In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/pure_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/pure_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [3]:
phys_net = PhysNet(norm='layer')
phys_net.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:54630323


In [9]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet-LN_PURE.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet-LN_PURE.h5')

Epoch 1/3
25/25 [==============================] - 30s 818ms/step - loss: 0.4257 - val_loss: 0.1465

Epoch 00001: val_loss improved from inf to 0.14645, saving model to ../weights\PhysNet-LN_PURE.h5
Epoch 2/3
25/25 [==============================] - 17s 573ms/step - loss: 0.1693 - val_loss: 0.1125

Epoch 00002: val_loss improved from 0.14645 to 0.11250, saving model to ../weights\PhysNet-LN_PURE.h5
Epoch 3/3
25/25 [==============================] - 17s 572ms/step - loss: 0.1264 - val_loss: 0.0982

Epoch 00003: val_loss improved from 0.11250 to 0.09824, saving model to ../weights\PhysNet-LN_PURE.h5


In [4]:
phys_net.load_weights('../weights/PhysNet-LN_PURE.h5')

In [5]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_PURE_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_PURE_UBFC.h5')

100%|██████████| 42/42 [00:23<00:00,  1.75it/s]


{'Sliding window': {'MAE': 1.413, 'RMSE': 3.684, 'R': 0.97707},
 'Whole video': {'MAE': 0.898, 'RMSE': 1.398, 'R': 0.9971}}